In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
#import lightgbm as lbgm
#from xgboost import XGBRegressor
#import xgboost as xgb
from catboost import CatBoostRegressor
#from lightgbm.sklearn import lbgmRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc

In [2]:
sys.path.append(os.path.realpath('..'))

In [3]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [4]:
del train_df['Over18']

In [5]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [6]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [7]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [7]:
def cat_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

    param_grid = {
        'depth': trial.suggest_int('depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        #'rsm': trial.suggest_float('rsm', 0.001, 0.9),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100), 
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 20),
        'random_strength': trial.suggest_float('random_strength', 0.001, 0.9),
    } 

    clf = CatBoostRegressor(iterations=20000,
                            **param_grid,
                            bootstrap_type='Bernoulli',
                            grow_policy='SymmetricTree',
                            #loss_function='Logloss',
                            eval_metric='AUC',
                            task_type="GPU",
                            random_state=1,)
    
    clf.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)
    preds = clf.predict(X_valid)

    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    cat_ranks[roc_auc] = clf

    return roc_auc

In [8]:
cat_ranks = {}

train_time = 1 * 60 * 5
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='CATRegressor')
study.optimize(cat_objective, timeout=train_time)



[I 2023-01-16 14:49:21,423] A new study created in memory with name: CATRegressor
Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8052718	best: 0.8052718 (0)	total: 8.93ms	remaining: 2m 58s


[I 2023-01-16 14:49:23,758] Trial 0 finished with value: 0.884584845544049 and parameters: {'depth': 2, 'learning_rate': 0.0692255083886799, 'subsample': 0.2640157395355086, 'min_data_in_leaf': 69, 'l2_leaf_reg': 17, 'random_strength': 0.6128310305465501}. Best is trial 0 with value: 0.884584845544049.


bestTest = 0.8845848455
bestIteration = 39
Shrink model to first 40 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

bestTest = 0.8912609093
bestIteration = 65
Shrink model to first 66 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8750092	best: 0.8750092 (0)	total: 19ms	remaining: 6m 20s


[I 2023-01-16 14:55:34,672] Trial 45 finished with value: 0.8855731562029205 and parameters: {'depth': 7, 'learning_rate': 0.09467393125976004, 'subsample': 0.5034462571290992, 'min_data_in_leaf': 84, 'l2_leaf_reg': 12, 'random_strength': 0.5730725072525309}. Best is trial 15 with value: 0.900891466316285.


bestTest = 0.8855731562
bestIteration = 12
Shrink model to first 13 iterations.


In [9]:
cat_ranks_bytes = pickle.dumps(cat_ranks)
with open("rendered_data/CAT_clsfs_bytes.hex", "wb") as binary_file:
    binary_file.write(cat_ranks_bytes)

In [14]:
percent = math.ceil(len(cat_ranks.keys()) * .1)
top = sorted(list(cat_ranks.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    train_preds.append(cat_ranks[key].predict(X))
    test_preds.append(cat_ranks[key].predict(X_test))
    
train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 249.99it/s]


In [15]:
CAT_train_submission = pd.DataFrame(data={'id': train_df.id, 'stroke': train_final_preds})
CAT_train_submission.to_csv(fr'rendered_data/CAT_train_submission.csv', index=False)

CAT_test_submission = pd.DataFrame(data={'id': test_df.id, 'stroke': test_final_preds})
CAT_test_submission.to_csv(fr'rendered_data/CAT_test_submission.csv', index=False)